<a href="https://colab.research.google.com/github/patrickhaddadteaching/testfromcolab/blob/main/test_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import wave
from IPython.display import display, Audio
from os import system

In [4]:
music_file_name='%d.wav'%(np.random.randint(0,10,dtype=np.uint8))
system('wget http://patrickhaddad.xyz/data/wavs/%s'%(music_file_name))

0

In [5]:
obj = wave.open(music_file_name,'r')
Number_of_frames=obj.getnframes()
sampwidth=obj.getsampwidth()
sampleRate = obj.getframerate()
v_frames=np.frombuffer(obj.readframes(Number_of_frames),dtype=np.uint8)
obj.close()

In [6]:
sounddata=np.frombuffer(v_frames.tobytes(),dtype='int64')
ret_audio=Audio(sounddata,rate=8000)
display(ret_audio)